In [46]:
"""SNAILFISH HOMEWORK 2021 Day 18 AoC https://adventofcode.com/2021/day/18"""

import ast
import math

class Tree():
    """Tree.d will contain info about depth"""
    def __init__(self):
        self.d={}
        self.stack=[]

    def l2d(self, s):
        """list to dictionary. Uncomment for debug"""
        #print("list:",s)
        #print("dict:", self.d)
        #print("stack:",self.stack)
        
        if isinstance(s[0], int) and isinstance(s[1], int):
            self.d["".join(self.stack)+"0"] = int(s[0])
            self.d["".join(self.stack)+"1"] = int(s[1])

        elif isinstance(s[1], int):
            #print(s)
            self.d["".join(self.stack)+"1"] = int(s.pop())
            self.stack.append("0")
            self.l2d(s[0])
            self.stack.pop()

        elif isinstance(s[0], int):
            self.d["".join(self.stack)+"0"] = int(s.pop(0))
            self.stack.append("1")
            self.l2d(s[0]) #after popping there's only 1 element in the list [[x,y]] that has to be accessed always with [0] only
            self.stack.pop()

        else:
            self.stack.append("0")
            self.l2d(s[0])
            self.stack.pop()

            self.stack.append("1")
            self.l2d(s[1])
            self.stack.pop()

def ex(d):
    """explode next element in dictionary"""
    leaves = sorted(d)
    for i,lf in enumerate(sorted(d)):
        if len(lf) > 4:
            if lf[-1] == '0':

                if i != 0:
                    d[str(leaves[i-1])]+=d[str(leaves[i])]
                d[str(leaves[i])[:-1]]=0
                del d[str(leaves[i])]

                if i+1 != len(leaves)-1: #explode to the right if we're not at the end
                    d[str(leaves[i+2])]+=d[str(leaves[i+1])]
                d[str(leaves[i+1])[:-1]]=0
                del d[str(leaves[i+1])]
                break
    return d

def sp(d):
    """split next element in dictionary"""
    for i in sorted(d):
        if d[i] > 9:
            d[i+"0"] = d[i]//2
            d[i+"1"] = math.ceil(d[i]/2)
            del d[i]
            break            
    return d

def red(d):
    """reduce snailfishnumber. Uncomment for debug"""
    while True:
        #print(d)
        #input()
        if any(len(i)>4 for i in d):
            d = ex(d)
            continue           
        if any(d[i]>9 for i in d):
            d = sp(d)
            continue            
        break
    return d

def add(d1, d2):
    """add snailfishnumbers as dictionary"""
    d1out = {}
    for i in d1:
        d1out["0"+i] = d1[i]
        
    d2out = {}
    for i in d2:
        d2out["1"+i] = d2[i]
        
    d1out |= d2out
    return d1out

def mg(d):
    """calculate magnitude of snailfishnumber"""
    for i in reversed(range(4)): #get the deepest level
        leaves = sorted(k for k in d if len(k)==i+1)
        if leaves == []:
            continue
        lf = iter(leaves)
        while True:
            try:
                add1 = next(lf)
                res = d[add1]*3 + d[next(lf)]*2
                d[add1[:-1]] = res
            except StopIteration:
                break
                
    return d[""]

def test_red():
    """test if reduction works with given examples"""
    i1,i2,o = "[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]", "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"
    T1 = Tree()
    T1.l2d(ast.literal_eval(i1))
    T2 = Tree()
    T2.l2d(ast.literal_eval(i2))
    T3 = Tree()
    T3.l2d(ast.literal_eval(o))

    pred = red(add(T1.d, T2.d))
    if pred != T3.d:
        return f"problem with adding {i1} and {i2}. Should be {o} but is {pred}."

    out = ["[[[[5,0],[7,4]],[5,5]],[6,6]]", "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]",
           "[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]"]
    
    for i,o in zip(["i1", "i2", "i3"], out):
        addl = []
        with open(i, "r") as f:
            for l in f:
                addl.append(l.strip("\n"))

        T1 = Tree()
        T1.l2d(ast.literal_eval(addl[0]))

        for snailnr in addl:
            T2 = Tree()
            T2.l2d(ast.literal_eval(snailnr))
            T1.d = red(add(T1.d, T2.d))

        T3 = Tree()
        T3.l2d(ast.literal_eval(o))

        if T1.d != T3.d:
            return f"Problem with adding numbers in file {i}.\nShould be:\n{T3.d}\nbut is:\n{T1.d}"

    return "All tests passed"
    
def test_mg():
    """test if magnitude works with given examples"""
    in_ = ["[9,1]", "[[9,1],[1,9]]", "[[1,2],[[3,4],5]]", "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]",
           "[[[[1,1],[2,2]],[3,3]],[4,4]]", "[[[[3,0],[5,3]],[4,4]],[5,5]]",
           "[[[[5,0],[7,4]],[5,5]],[6,6]]", "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]"]
    out = [29, 129, 143, 1384, 445, 791, 1137, 3488]
    for i,o in zip(in_, out):
        T1 = Tree()
        T1.l2d(ast.literal_eval(i))
        pred = mg(T1.d)
        if pred != o:
            return f"problem with {i}. Should be {o} but is: {pred}"
    
    return "All tests passed"

def test():
    """test if all tests pass"""
    if test_mg() != 'All tests passed':
        return "problem with magnitude. Run test_mg()"
    if test_red() != 'All tests passed':
        return "problem with reduction. Run test_red()"
    return 'All tests passed'

test()

'problem with reduction. Run test_red()'

In [47]:
print(test_red())

Problem with adding numbers in file i3.
Should be:
{'0000': 6, '0001': 6, '0010': 7, '0011': 6, '0100': 7, '0101': 7, '0110': 7, '0111': 0, '1000': 7, '1001': 7, '1010': 7, '1011': 7, '1100': 7, '1101': 8, '1110': 9, '1111': 9}
but is:
{'0000': 6, '0001': 6, '0010': 7, '0011': 7, '0100': 7, '0101': 7, '0110': 7, '0111': 7, '1111': 7, '1110': 8, '1101': 8, '1100': 7, '1011': 7, '1010': 7, '1001': 7, '1000': 0}


In [48]:
#the same problem
#debug here

addl = []
with open("i3", "r") as f:
    for l in f:
        addl.append(l.strip("\n"))

T1 = Tree()
T1.l2d(ast.literal_eval(addl[0]))

for snailnr in addl:
    T2 = Tree()
    T2.l2d(ast.literal_eval(snailnr))
    T1.d = red(add(T1.d, T2.d))

T3 = Tree()
T3.l2d(ast.literal_eval("[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]"))

print(T3.d)
print(T1.d)

print(f"correct magnitude: 4140 (calculated: {mg(T3.d)})")
print("predicted: magnitude:", mg(T1.d))

{'0000': 6, '0001': 6, '0010': 7, '0011': 6, '0100': 7, '0101': 7, '0110': 7, '0111': 0, '1000': 7, '1001': 7, '1010': 7, '1011': 7, '1100': 7, '1101': 8, '1110': 9, '1111': 9}
{'0000': 6, '0001': 6, '0010': 7, '0011': 7, '0100': 7, '0101': 7, '0110': 7, '0111': 7, '1111': 7, '1110': 8, '1101': 8, '1100': 7, '1011': 7, '1010': 7, '1001': 7, '1000': 0}
correct magnitude: 4140 (calculated: 4140)
predicted: magnitude: 3910
